# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 

#print(len(full_data_rows_list))
#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding 

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
query = "CREATE TABLE IF NOT EXISTS song_by_session"
query = query + "(session_id int, item_in_session int, artist_name text, song text, length decimal, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:        
        query = "INSERT INTO song_by_session (session_id, item_in_session, artist_name, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))


### Print the artist, song title and song's length during sessionId = 338, and itemInSession  = 4

In [10]:
query_1 = "SELECT artist_name,song,length FROM song_by_session WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)
    
table_1 = PrettyTable()
table_1.field_names = ["artist_name", "song", "length"]
for row in rows:
    table_1.add_row([row.artist_name, row.song, row.length])

print(table_1)

+-------------+---------------------------------+----------+
| artist_name |               song              |  length  |
+-------------+---------------------------------+----------+
|  Faithless  | Music Matters (Mark Knight Dub) | 495.3073 |
+-------------+---------------------------------+----------+


In [11]:
query = "CREATE TABLE IF NOT EXISTS song_by_UserSession "
query = query + "(user_id int, session_id int, item_in_session int, first_name text, last_name text, artist_name text, song text, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
       
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_UserSession(user_id, session_id, item_in_session, first_name, last_name, artist_name, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))



### Print name of artist, user and song for userid = 10, sessionid = 182

In [12]:
query_2 = "SELECT artist_name,first_name,last_name,song FROM song_by_UserSession WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
table_2 = PrettyTable()
table_2.field_names = ["artist_name", "first_name", "last_name", "song"]
for row in rows:
    table_2.add_row([row.artist_name, row.first_name, row.last_name, row.song])

print(table_2)

+-------------------+------------+-----------+------------------------------------------------------+
|    artist_name    | first_name | last_name |                         song                         |
+-------------------+------------+-----------+------------------------------------------------------+
|  Down To The Bone |   Sylvie   |    Cruz   |                  Keep On Keepin' On                  |
|    Three Drives   |   Sylvie   |    Cruz   |                     Greece 2000                      |
| Sebastien Tellier |   Sylvie   |    Cruz   |                      Kilometer                       |
|   Lonnie Gordon   |   Sylvie   |    Cruz   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |
+-------------------+------------+-----------+------------------------------------------------------+


In [13]:
query = "CREATE TABLE IF NOT EXISTS user_by_song "
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
        
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO user_by_song (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
                    

### Print user names who listened to the song 'All Hands Against His Own'


In [14]:
query_3 = "SELECT first_name,last_name,song FROM user_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)


table_3 = PrettyTable()
table_3.field_names = ["first_name", "last_name"]
for row in rows:
    table_3.add_row([row.first_name, row.last_name])

print(table_3)


+------------+-----------+
| first_name | last_name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop tables

In [15]:
query = "drop table song_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_by_UserSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()